In [107]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
import matplotlib.pyplot as plt

import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder


from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from skimage import io
import sklearn
from sklearn.model_selection import train_test_split
import os


In [108]:
img = Image.open(r'C:\Users\marth\Downloads\archive\US_Pictures\US_Pictures\99.10 App.bmp')
img.show()
img_dir = r'C:\Users\marth\Downloads\archive\US_Pictures\US_Pictures'


In [109]:
diag_data= pd.read_csv(r'C:\Users\marth\Downloads\archive\app_data.csv', encoding='latin1')
diag_data.head()


,Age,BMI,Sex,Height,Weight,Length_of_Stay,Management,Severity,Diagnosis_Presumptive,Diagnosis,...,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68
0,0.00,17.10,male,164.0,46.00,6.0,primary surgical,uncomplicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.04,14.10,male,53.0,3.96,12.0,primary surgical,complicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.53,20.37,male,176.0,63.10,3.0,primary surgical,uncomplicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.85,18.49,male,143.0,37.80,3.0,conservative,uncomplicated,appendicitis,no appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.73,17.93,male,83.5,12.50,3.0,conservative,uncomplicated,no appendicitis,no appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
#Get the diagnoses one hot encoded
diag_data = diag_data[diag_data["Appendix_on_US"] == r'yes'] # Remove entries that don't have an ultrasound of the appendix
present = diag_data["US_Number"]


diagnosis = (diag_data["Diagnosis"] == "appendicitis").astype(int)
print(diagnosis)

diag_data.head(-20 )

0      1
1      1
2      1
3      0
4      0
      ..
774    1
775    1
777    1
778    0
779    1
Name: Diagnosis, Length: 504, dtype: int64


,Age,BMI,Sex,Height,Weight,Length_of_Stay,Management,Severity,Diagnosis_Presumptive,Diagnosis,...,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68
0,0.00,17.10,male,164.0,46.00,6.0,primary surgical,uncomplicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.04,14.10,male,53.0,3.96,12.0,primary surgical,complicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.53,20.37,male,176.0,63.10,3.0,primary surgical,uncomplicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.85,18.49,male,143.0,37.80,3.0,conservative,uncomplicated,appendicitis,no appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.73,17.93,male,83.5,12.50,3.0,conservative,uncomplicated,no appendicitis,no appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,16.66,21.50,male,174.0,65.00,5.0,conservative,uncomplicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,16.68,20.30,female,169.0,58.00,3.0,conservative,uncomplicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
740,16.72,18.50,male,180.0,60.00,3.0,conservative,uncomplicated,appendicitis,no appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,16.77,25.40,female,166.0,70.00,3.0,conservative,uncomplicated,appendicitis,appendicitis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

images = [f for f in os.listdir(img_dir) if f.lower().endswith('.bmp')]

image_df = pd.DataFrame(images, columns=['filename'])

image_df['patient_id'] = image_df['filename'].str.split('.').str[0].astype(int)

image_df.head()

diagnosis_map = diag_data.set_index("US_Number")["Diagnosis"]
print(diagnosis_map)

image_df["Appendix_on_US"] = image_df["patient_id"].map(diagnosis_map)


image_df.head()


#print(diag_data['patient_id'])

#image_df.head(-5)


US_Number
205.0       appendicitis
260.0       appendicitis
198.0       appendicitis
376.0    no appendicitis
371.0    no appendicitis
              ...       
114.0       appendicitis
756.0       appendicitis
735.0       appendicitis
922.0    no appendicitis
188.0       appendicitis
Name: Diagnosis, Length: 504, dtype: object


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [116]:


class MedicalDataset(Dataset):
    def __init__(self, img_dir, transform=None):

        self.data = [f for f in os.listdir(img_dir) if f.lower().endswith('.bmp')] #eliminates pngs and other filetypes
        self.img_dir = img_dir
        self.transform = transform

        #not sure what this is
        self.classes = diagnosis
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data[idx])
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
            return image
        
    def getpath(self, idx):
        img_path = os.path.join(self.img_dir, self.data[idx])
        return img_path
        
       
    
# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    #transforms.Normalize([0.5], [0.5])
])

# Create dataset and loader
dataset = MedicalDataset(
    img_dir=r'C:\Users\marth\Downloads\archive\US_Pictures\US_Pictures',
    transform=transform
)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Check batch
images = next(iter(loader))
print(images.shape)

check = dataset.__getitem__(54)
print(check)



torch.Size([16, 3, 224, 224])
tensor([[[0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1922, 0.1922, 0.1922,  ..., 0.1922, 0.1922, 0.1922],
         [0.1922, 0.1922, 0.1922,  ..., 0.1922, 0.1922, 0.1922],
         [0.1922, 0.1922, 0.1922,  ..., 0.1922, 0.1922, 0.1922],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.2471, 0.2471, 0.2471,  ..., 0.2471, 0.2471, 0.2471],
         [0.2471, 0.2471, 0.2471,  ..., 0.2471, 0.2471, 0.2471],
         [0.2471, 0.2471, 0.

In [ ]:

answers = 0

def chicken():
    for x in dataset:
        x.getpath()
        for y in present:
            print(y)
            if y.str.contains(x):
                answers += answers
    print(answers)

chicken()
            

AttributeError: 'Tensor' object has no attribute 'getpath'

In [64]:
X_train, x_test, y_train, y_test = train_test_split( dataset, diagnosis, test_size=.8, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [2064, 782]

In [ ]:
from torchvision.io import decode_image
from torchvision.models import resnet50, ResNet50_Weights

epochs = 5

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.BCELoss()  # Suitable for classification
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)


# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(dataset.__getitem__())

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).binarycrossentropy(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

TypeError: MedicalDataset.__getitem__() missing 1 required positional argument: 'idx'